In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [2]:
from sklearn import metrics

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import Birch
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MiniBatchKMeans

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")


In [5]:
df = pd.read_csv('data/sports.csv')

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Unnamed: 0,text,class,id
0,1527,#polo \nU know polo??,3,2407
1,2382,#Soccer #Basketball #UFC #MMA #Tennis\n#WaterP...,4,3262
2,91,Rees Mogg is like a metronome...that started t...,0,971
3,41,Q10. I am a country that will be hosting India...,0,921
4,1472,Why is Paul Azinger on @NBCGolf and @FOXGOLF?\...,1,2352


In [7]:
df.shape

(2500, 4)

In [8]:
from sklearn import preprocessing

label_encoding = preprocessing.LabelEncoder()

df['class'] = label_encoding.fit_transform(df['class'].astype(str))

df.head()

,Unnamed: 0,text,class,id
0,1527,#polo \nU know polo??,3,2407
1,2382,#Soccer #Basketball #UFC #MMA #Tennis\n#WaterP...,4,3262
2,91,Rees Mogg is like a metronome...that started t...,0,971
3,41,Q10. I am a country that will be hosting India...,0,921
4,1472,Why is Paul Azinger on @NBCGolf and @FOXGOLF?\...,1,2352


In [9]:
df_labels = df['class']

df_labels.sample(10)

548     0
797     2
1037    2
432     2
1438    1
1057    4
1375    1
2241    1
1600    1
1906    3
Name: class, dtype: int64

In [10]:
def build_model(clustering_model, data, labels):
    
    model = clustering_model(data)

    print('homo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    print(50 * '-')
    
    print('%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          %(metrics.homogeneity_score(labels, model.labels_),
            metrics.completeness_score(labels, model.labels_),
            metrics.v_measure_score(labels, model.labels_),
            metrics.adjusted_rand_score(labels, model.labels_),
            metrics.adjusted_mutual_info_score(labels,  model.labels_),
            metrics.silhouette_score(data, model.labels_)))

In [11]:
def k_means(data, n_clusters=5, max_iter=1000):
    model = KMeans(n_clusters=n_clusters, max_iter=max_iter).fit(data)
    
    return model

In [12]:
def agglomerative_fn(data, n_clusters=5):
    model = AgglomerativeClustering(n_clusters = n_clusters).fit(data)
    
    return model

In [13]:
def dbscan_fn(data, eps=0.45, min_samples=4):
    model = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    return model

In [14]:
def mean_shift_fn(data, bandwidth=0.85):
    model = MeanShift(bandwidth=bandwidth).fit(data)
    return model

In [15]:
def birch_fn(data, n_clusters=5):
    model = Birch(n_clusters=n_clusters).fit(data)
    return model

In [16]:
def affinity_propagation_fn(data, damping=0.6, max_iter=1000):
    model = AffinityPropagation(damping=damping, max_iter=max_iter).fit(data)
    return model

In [17]:
def mini_batch_kmeans_fn(data, n_clusters=5, max_iter=1000):
    model = MiniBatchKMeans(n_clusters=n_clusters, max_iter=max_iter, batch_size=20).fit(data)
    return model

### TFIDF

In [16]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english')
df_tfidf=tfidf_vectorizer.fit_transform(df['text']) 

In [17]:
df_tfidf

<2500x10666 sparse matrix of type '<class 'numpy.float64'>'
	with 35853 stored elements in Compressed Sparse Row format>

In [18]:
build_model(k_means, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.720	0.803	0.759	0.594	0.758	0.022


In [19]:
build_model(agglomerative_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.218	0.521	0.308	0.088	0.306	0.020


In [20]:
build_model(dbscan_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.008	0.229	0.016	0.000	0.012	0.004


In [21]:
build_model(mean_shift_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
1.000	0.210	0.347	0.002	0.034	0.049


In [22]:
build_model(birch_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.342	0.503	0.407	0.226	0.406	0.018


In [23]:
build_model(affinity_propagation_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.730	0.216	0.333	0.037	0.272	0.050


In [24]:
build_model(mini_batch_kmeans_fn, df_tfidf.toarray(), df_labels)

homo	compl	v-meas	ARI	AMI	silhouette
--------------------------------------------------
0.370	0.645	0.471	0.219	0.469	0.015


### Doc2Vec

In [19]:
from getEmbeddings import getEmbeddings
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import scikitplot.plotters as skplt

def plot_cmat(yte, ypred):
    '''Plotting confusion matrix'''
    skplt.plot_confusion_matrix(yte,ypred)
    plt.show()


x,y = getEmbeddings('data/sports.csv')

In [26]:
def doc2vec_build_model(clustering_model, data, labels):
    
    model = clustering_model(data)

    print('homo\tcompl\tv-meas\tARI\tAMI')
    print(50 * '-')
    
    print('%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          %(metrics.homogeneity_score(labels, model.labels_),
            metrics.completeness_score(labels, model.labels_),
            metrics.v_measure_score(labels, model.labels_),
            metrics.adjusted_rand_score(labels, model.labels_),
            metrics.adjusted_mutual_info_score(labels,  model.labels_)))

In [27]:
doc2vec_build_model(k_means, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.043	0.046	0.045	0.026	0.043


In [28]:
doc2vec_build_model(agglomerative_fn, x,y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.036	0.046	0.040	0.022	0.038


In [29]:
doc2vec_build_model(dbscan_fn, x,y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
-0.000	1.000	-0.000	0.000	-0.000


In [30]:
doc2vec_build_model(mean_shift_fn, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
-0.000	1.000	-0.000	0.000	-0.000


In [31]:
doc2vec_build_model(birch_fn,x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
-0.000	1.000	-0.000	0.000	-0.000


In [32]:
doc2vec_build_model(affinity_propagation_fn, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.109	0.036	0.054	0.002	0.017


In [33]:
doc2vec_build_model(mini_batch_kmeans_fn, x, y)

homo	compl	v-meas	ARI	AMI
--------------------------------------------------
0.036	0.052	0.042	0.028	0.040
